In [1]:
{-# LANGUAGE RankNTypes 
              #-}

Refs:
- (https://wiki.haskell.org/State_Monad)
- (https://en.wikibooks.org/wiki/Haskell/Understanding_monads/State)
- [Faster state monad](https://www.reddit.com/r/haskell/comments/1rcc8t/performance_of_the_st_monad_over_the_state_monad/)
- (http://lpaste.net/96126)
- (https://wiki.haskell.org/Monad/ST)


In [7]:
import Control.Monad.State.Strict
import Control.Monad.ST
import Data.STRef
import qualified Data.HashMap.Strict as Map
import qualified Data.Foldable as F

sum' = F.foldl' (+) 0

exampleState xs = flip evalState Map.empty $ do
  F.forM_ xs $ \x -> do
    when (x `rem` 10000 == 0) $ modify' (Map.insert x x)
  gets $ sum' . Map.elems
  where modify' f = do v <- get
                       put $! f v
:t runST
exampleST xs = runST $ do
  state <- newSTRef Map.empty
  F.forM_ xs $ \x -> do
    when (x `rem` 10000 == 0) $ modifySTRef' state (Map.insert x x)
    
  state' <- readSTRef state
  return $ sum' $ Map.elems state'

print $ exampleState [(1::Integer)..1000000]
--main = print $ exampleState [(1::Integer)..100000000]
-- main = print $ exampleST [(1::Integer)..100000000]


-- older versions of base do not have modifySTRef':

-- modifySTRef' :: STRef s a -> (a -> a) -> ST s ()
-- modifySTRef' ref f = do
--     x <- readSTRef ref
--     let x' = f x
--     x' `seq` writeSTRef ref x'

Line 3: Redundant do
Found:
do when (x `rem` 10000 == 0) $ modifySTRef' state (Map.insert x x)
Why not:
when (x `rem` 10000 == 0) $ modifySTRef' state (Map.insert x x)

runST :: forall a. (forall s. ST s a) -> a

50500000